In [ ]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import getpass
import os


os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

MISTRAL_API_URL = "https://api.mistral.ai/v1/chat/completions"  

# Schéma d’état
class EtatIA(TypedDict):
    question: str
    reponse: Optional[str]

# Initialiser le modèle
model = init_chat_model("mistral-large-latest", model_provider="mistralai")

# Nœud du graphe : réponse à une question
def noeud_reponse(state: EtatIA) -> EtatIA:
    question = state["question"]
    result = model.invoke(question)  # PAS besoin de HumanMessage ici
    return {
        "question": question,
        "reponse": result.content
    }

# Construction du graphe
graph = StateGraph(EtatIA)
graph.add_node("repondre", noeud_reponse)
graph.set_entry_point("repondre")
graph.add_edge("repondre", END)
app = graph.compile()

# Exécution du graphe
etat_initial: EtatIA = {
    "question": "Bonjour, peux-tu me résumer ce qu’est l’intelligence artificielle ?",
    "reponse": None
}
etat_final = app.invoke(etat_initial)
print("Réponse :", etat_final["reponse"])